In [2]:
import pandas as pd
import numpy as np
import json
from scipy import sparse as sp
from tqdm.notebook import tqdm
from collections import defaultdict

In [4]:
!pip install implicit

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 1.1 MB 97 kB/s eta 0:00:011     |██████████▋                     | 368 kB 923 kB/s eta 0:00:01
  Created wheel for implicit: filename=implicit-0.4.2-cp37-cp37m-linux_x86_64.whl size=3101814 sha256=deca4c35e0e276edcb12c09b82faa5f0f8dc673f24baed4fe992fed8a69563ab
  Stored in directory: /home/jovyan/.cache/pip/wheels/2f/ed/a0/bcad2756c8a99e49d6edaf830ab604e5691910df4dd7cec870
Successfully built implicit


In [5]:
import implicit

In [6]:
import sys
sys.path.append('../')

from src.utils import get_shard_path
from src.utils import ProductEncoder, make_coo_row
from src.metrics import normalized_average_precision

In [17]:
def make_coo_row(transaction_history, product_encoder: ProductEncoder):
    idx = []
    values = []

    items = []
    for trans in transaction_history:
        items.extend([i["product_id"] for i in trans["products"]])
    n_items = len(items)

    for pid in items:
        idx.append(product_encoder.toIdx(pid))
        values.append(1.0)  # убираем нормировку
        

    return sp.coo_matrix(
        (np.array(values).astype(np.float32), ([0] * len(idx), idx)), shape=(1, product_encoder.num_products),
    )

In [18]:
product_encoder = ProductEncoder('../data/raw/products.csv')

In [19]:
valid_data = [json.loads(l) for l in open(get_shard_path(7))][:3000]

In [20]:
rows = []
for shard_id in range(4):
    for js in tqdm(json.loads(l) for l in open(get_shard_path(shard_id))):
        rows.append(make_coo_row(js["transaction_history"], product_encoder))

In [21]:
X_sparse = sp.vstack(rows).tocsr()

In [22]:
X_sparse.shape

(40421, 43038)

# ALS

In [23]:
model = implicit.als.AlternatingLeastSquares(factors=16, regularization=0.0, iterations=8)
model.fit(X_sparse.T)  # обязательный шаг

In [24]:
m_ap = []
for js in tqdm(valid_data):
    row_sparse = make_coo_row(js["transaction_history"], product_encoder).tocsr()
    raw_recs = model.recommend(0, row_sparse, N=30, filter_already_liked_items=False, recalculate_user=True)
    recommended_items = product_encoder.toPid([x[0] for x in raw_recs])
    gt_items = js["target"][0]["product_ids"]
    m_ap.append(normalized_average_precision(gt_items, recommended_items, k=30))
print(np.mean(m_ap))


0.07104098365878458
